In [1]:
import findspark
findspark.init()

In [2]:
import yaml
import os

import pandas as pd
import numpy as np
import tensorflow as tf

from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType
from pyspark.sql.functions import col, to_date, to_timestamp, explode
from pyspark.sql.utils import AnalysisException
from pyspark.errors import PySparkException
from pymongo import MongoClient
from pymongo import errors

2024-03-09 04:40:34.102693: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
/home/sutring/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def gen_mongo_uri(mongo_conf) -> str | None:
    """
    generate mongo connection uri based on input
    :return:
    """
    if not mongo_conf:
        return None
    return (f"mongodb+srv://{mongo_conf['user']}:"
            f"{mongo_conf['token']}@"
            f"{mongo_conf['host']}"
            f"/?retryWrites=true&w=majority")

def get_property(conf) -> dict:
    """
    get property for database
    """
    return {key: conf[key] for key in conf.keys()
                         & {'user', 'password', 'driver'}}

def gen_maria_jdbc(conf, db: str = '') -> str | None:
    """
    get connection jdbc string for maria database
    """
    if not conf:
        return None

    db = conf['database'] if not db else db
    return (f"jdbc:mysql://{conf['host']}:"
            f"{conf['port']}/"
            f"{db}?permitMysqlScheme")

def gen_postgres_jdbc(conf, db: str = '') -> str | None:
    """
    get connection jdbc string for maria database
    """
    if not conf:
        return None
    db = conf['database'] if not db else db
    return (f"jdbc:postgresql://{conf['host']}:"
            f"{conf['port']}/{db}")

def gen_mssql_jdbc(conf, db: str = '') -> str | None:
    """
    get connection jdbc string for maria database
    """
    if not conf:
        return None
    db = conf['database'] if not db else db
    return (f"jdbc:sqlserver://{conf['host']}:{conf['port']};"
            f"databaseName={db};encrypt=true;trustServerCertificate=true;")

def gen_azure_jdbc(conf, db: str = '') -> str | None:
    """
    get connection jdbc string for azure sql database
    """
    if not conf:
        return None
    db = conf['database'] if not db else db
    return (f"jdbc:sqlserver://{conf['host']}:{conf['port']};"
            f"databaseName={db};encrypt=true;")


def init_mongodb_client(uri: str) -> MongoClient | None:
    """
    initialize the mongo client
    """
    try:
        # Initialize new MongoDB client
        client = MongoClient(uri)
    except errors.ConnectionFailure as e:
        # Handle connection failure gracefully
        print(f"Failed to connect to MongoDB: {e}")

        return None
    else:
        return client

def prepare_dataframe(spark, sc, data) -> DataFrame:
    """
    prepare dataframe when the data might be in different data types
    :param spark: spark session
    :param sc: spark context
    :param data: data to be processed
    :return: spark sql dataframe
    """
    if isinstance(data, pd.DataFrame) and not data.empty:
        df = spark.createDataFrame(data)
    elif isinstance(data, DataFrame) and not data.isEmpty():
        df = data
    elif isinstance(data, list):
        df = spark.read.json(sc.parallelize([json.dumps(record) for record in data]))
    elif isinstance(data, dict):
        data = [data]
        df = spark.read.json(sc.parallelize([json.dumps(record) for record in data]))
    else:
        # initialize empty dataframe
        schema = StructType([])
        df = spark.createDataFrame([], schema)

    return df
    
def write_to_mongo(spark, data, 
                   uri: str, db: str, col: str) -> None:
    """
    write data into mongo db
    :param spark: sparkSession
    :param data: 
    :param uri: str, 
    :param db: str, 
    :param col: str, 
    :return:
    """
    sc = spark.sparkContext
    df = prepare_dataframe(spark, sc, data)
    config = {
        'uri': uri,
        'database': db,
        'collection': col
    }
    if not df.isEmpty():
        df.write.format("mongo") \
            .options(**config) \
            .mode("append") \
            .save()

def write_to_database(spark, data, conf,
                   db: str, write_table: str, type: str = 'mariadb') -> None:
    """
    write data into database table
    :param data:
    :param uri:
    :param db:
    :param table:
    :param type:
    :return:
    """
    sc = spark.sparkContext
    df = prepare_dataframe(spark, sc, data)
    properties = get_property(conf)

    jdbc = ''
    if type == 'mariadb':
        jdbc = gen_maria_jdbc(conf, db)
    elif type == 'postgres':
        jdbc = gen_postgres_jdbc(conf, db)
    elif type == 'mssql':
        jdbc = gen_mssql_jdbc(conf, db)
    elif type == 'azure':
        jdbc = gen_azure_jdbc(conf, db)
    else:
        raise ValueError('error when generating jdbc')

    if not df.isEmpty():
        df.write.jdbc(
            url=jdbc,
            table=write_table,
            mode="append",
            properties=properties
        )
    else:
        print('empty dataset')

def sentiment_analysis(data, tokenizer, model) -> pd.DataFrame:
    """
    sentiment analysis and retrun dataframe with score
    """
    data['sentiment'] = None
    for idx, text in enumerate(data['text']):
        tokenized_news = tokenizer(text, truncation=True, return_tensors="tf")
        logits = model.predict(tokenized_news).logits
        probabilities = tf.nn.softmax(logits)
        data.at[idx, 'sentiment'] = probabilities.numpy()
    return data

In [4]:
with open('conf.yaml', 'r') as file:
    config = yaml.safe_load(file)

mongo_uri = gen_mongo_uri(config['mongodb'])

In [5]:
try:
    # getting the spark instance
    spark = SparkSession.builder \
        .appName('Big Data Project ETL') \
        .config("spark.mongodb.input.uri", mongo_uri) \
        .config("spark.mongodb.output.uri", mongo_uri) \
        .config("spark.jars.packages", "com.microsoft.azure:spark-mssql-connector_2.12:1.2.0,"
                                       "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
        .master("local[*]") \
        .getOrCreate()
except PySparkException as e:
    print(f"Failed to get or create Spark: {e}")
except Exception as e:
    print(f"Exception Caught: {e}")

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml

In [6]:
# read historical data with sentiment from mariadb
historical_stock_df = spark.read.jdbc(
    url=gen_maria_jdbc(config['mariadb'], 'finance_out'),
    table='historical_with_sentiment',
    properties=get_property(config['mariadb'])
)
cot_report_df = spark.read.jdbc(
    url=gen_mssql_jdbc(config['mssql'], 'finance_api'),
    table='fmp_cmtmt_trader_report',
    properties=get_property(config['mssql'])
)
income_stmt_df = spark.read.jdbc(
    url=gen_postgres_jdbc(config['postgres'], 'finance_api'),
    table='fmp_income_stmt',
    properties=get_property(config['postgres'])
)

In [7]:
historical_stock_df.show(5)

+------+----------+-------------------+----------+---------+
|symbol|      date|              Close|    Volume|sentiment|
+------+----------+-------------------+----------+---------+
|  AAPL|2004-03-03|0.42714300751686096| 225131200|     NULL|
|  AAPL|2004-03-04|0.44928601384162903| 660223200|     NULL|
|  AAPL|2004-03-05|0.47749999165534973|1540599200|     NULL|
|  AAPL|2004-03-08| 0.4642859995365143| 522872000|     NULL|
|  AAPL|2004-03-09|0.48392900824546814| 618363200|     NULL|
+------+----------+-------------------+----------+---------+
only showing top 5 rows



In [8]:
cot_report_df.show(5)

+-------------------+------+--------------------------+---------------------------+-----------------------+------------------------+-----------+---------------+
|               date|symbol|noncomm_positions_long_all|noncomm_positions_short_all|comm_positions_long_all|comm_positions_short_all|comm_spread|non_comm_spread|
+-------------------+------+--------------------------+---------------------------+-----------------------+------------------------+-----------+---------------+
|2024-02-27 00:00:00|    BT|                     20034|                      22001|                   1759|                     799|        960|          -1967|
|2021-03-16 00:00:00|    BT|                      4797|                       7449|                    479|                     165|        314|          -2652|
|2024-02-20 00:00:00|    BT|                     19624|                      21722|                   1862|                     660|       1202|          -2098|
|2021-03-09 00:00:00|    BT|      

In [9]:
income_stmt_df.show(5)

+-------------------+------------+----------+---------------+-------------+----------+---------------------------+---------+------------+------+----------+-----------+--------------------+--------------------------------+-----------+----------------+---------------+--------------------+----------------+---------------+--------------+--------------------+---------+--------------+-----------------+---------------+--------------------+-------------+------+----------------+------------------------------+----------+---------------------------+---------------------------------------+------+---------------------------+---------------------+------------------------+
|       acceptedDate|calendarYear|       cik|costAndExpenses|costOfRevenue|      date|depreciationAndAmortization|   ebitda| ebitdaratio|   eps|epsdiluted|fillingDate|           finalLink|generalAndAdministrativeExpenses|grossProfit|grossProfitRatio|incomeBeforeTax|incomeBeforeTaxRatio|incomeTaxExpense|interestExpense|interestIncom

In [10]:
# get unique symbol
stock_symbols = historical_stock_df.select("symbol").distinct().rdd.flatMap(lambda x : x).collect()
sec_symbols = cot_report_df.select("symbol").distinct().rdd.flatMap(lambda x : x).collect()

In [23]:
# save historical data into mssql or azure, each stock as a table
for symbol in stock_symbols:
    stock_data = historical_stock_df.filter(historical_stock_df['symbol'] == symbol) \
                                .select('date', 'Close', 'sentiment') \
                                .withColumnRenamed('Close', symbol)
    try:
        # save with 'stock_' as prefix
        write_to_database(spark, stock_data, config['mssql'],
                          db='finance_out', write_table=f'stock_{symbol}', type='mssql')
    except Exception as e:
        print(e)

In [11]:
# save commitment report data into mssql or azure, each security as a table
for symbol in sec_symbols:
    cot_data = cot_report_df.filter(cot_report_df['symbol'] == symbol) 
    try:
        # save with 'stock_' as prefix
        write_to_database(spark, cot_data, config['mssql'],
                          db='finance_out', write_table=f'cot_report_{symbol}', type='mssql')
    except Exception as e:
        print(e)

In [12]:
# save income statement data into mssql or azure, each stock as a table
for symbol in stock_symbols:
    income_stmt_data = income_stmt_df.filter(income_stmt_df['symbol'] == symbol)
    try:
        # save with 'stock_' as prefix
        write_to_database(spark, income_stmt_data, config['mssql'],
                          db='finance_out', write_table=f'income_stmt_{symbol}', type='mssql')
    except Exception as e:
        print(e)

In [6]:
# load inside transactions from mongodb
inside_trading_df = spark.read.format('mongo') \
    .option("database", 'finance_api') \
    .option("collection", 'finn_insider_transactions') \
    .load()

inside_trading_df.show(5)

+--------------------+--------------------+------+
|                 _id|                data|symbol|
+--------------------+--------------------+------+
|{65e550a0b142680e...|[{-100000, , 2024...|  AAPL|
|{65e550bab142680e...|[{-300, , 2024-02...|  MSFT|
|{65e550d4b142680e...|[{-5000, , 2024-0...|  NVDA|
|{65e550ecb142680e...|[{-31493, , 2024-...|  META|
|{65e55104b142680e...|[{-500, , 2024-02...|  AMZN|
+--------------------+--------------------+------+
only showing top 5 rows



In [7]:
inside_trading_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- change: long (nullable = true)
 |    |    |-- currency: string (nullable = true)
 |    |    |-- filingDate: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- isDerivative: boolean (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- share: long (nullable = true)
 |    |    |-- source: string (nullable = true)
 |    |    |-- symbol: string (nullable = true)
 |    |    |-- transactionCode: string (nullable = true)
 |    |    |-- transactionDate: string (nullable = true)
 |    |    |-- transactionPrice: double (nullable = true)
 |-- symbol: string (nullable = true)



In [8]:
# flattern inside_trading schema
inside_trading_new_df = inside_trading_df.select('symbol', explode('data').alias('trade_info')) \
                                    .select('symbol', 'trade_info.transactionDate', 
                                            'trade_info.share', 'trade_info.name',
                                            'trade_info.transactionPrice', 'trade_info.change')
# df_flat = df_flat.select('name', 'age', 'car_info.car1', 'car_info.model')

inside_trading_new_df.show(15)

+------+---------------+-------+-----------------+----------------+-------+
|symbol|transactionDate|  share|             name|transactionPrice| change|
+------+---------------+-------+-----------------+----------------+-------+
|  AAPL|     2024-02-29|4434576|LEVINSON ARTHUR D|          180.94|-100000|
|  AAPL|     2024-02-28|   1516|     WAGNER SUSAN|             0.0|   1516|
|  AAPL|     2024-02-28|   1516|   SUGAR RONALD D|             0.0|   1516|
|  AAPL|     2024-02-28|   1516|  LOZANO MONICA C|             0.0|   1516|
|  AAPL|     2024-02-28|   1516|LEVINSON ARTHUR D|             0.0|   1516|
|  AAPL|     2024-02-28|   1516|      JUNG ANDREA|             0.0|   1516|
|  AAPL|     2024-02-28|   1516|      Gorsky Alex|             0.0|   1516|
|  AAPL|     2024-02-28|   1516|   Austin Wanda M|             0.0|   1516|
|  AAPL|     2024-02-01|      0|     WAGNER SUSAN|             0.0|  -1852|
|  AAPL|     2024-02-01|  60975|     WAGNER SUSAN|             0.0|   1852|
|  AAPL|    

In [10]:
inside_trading_new_df.count()

41787

In [12]:
# /user/input/project/fmp_company_profile
profile_df = spark.read.csv('hdfs://localhost:9000/user/input/project/fmp_company_profile', header=True)

In [13]:
profile_df.show(5)

+--------------------+-----+--------------------+-------+----------+----------+--------------------+-------+--------+---------+-------------------+---------+------------+--------------------+--------------------+-----------------+-----------------+--------------------+--------------------+----------+-----------------+-----+-----+------+------------+-------+-------------+------------+------+-------------+---------------+-----+------+--------+--------------------+----------+
|             address| beta|                 ceo|changes|       cik|      city|         companyName|country|currency|    cusip|                dcf|  dcfDiff|defaultImage|         description|            exchange|exchangeShortName|fullTimeEmployees|               image|            industry|   ipoDate|isActivelyTrading|isAdr|isEtf|isFund|        isin|lastDiv|       mktCap|       phone| price|        range|         sector|state|symbol|  volAvg|             website|       zip|
+--------------------+-----+----------------

In [ ]:
spark.stop()